In [1]:
import re
import pandas as pd
import os

In [5]:
CONFIG_DIR = "../linux-4.14.152"
# Possible options for generating a .config file
CONF_FLAG = ["randconfig", "defconfig", "allnoconfig",\
             "allyesconfig", "allmodconfig", "alldefconfig"]

def read_imp_val_csv(file):
    """Read impossible.csv like file
    :param: filename
    :type: string
    :return: dictionary associating param and set of impossible values
    :rtype: dict
    """
    df = pd.read_csv(file)
    df_dict = df.to_dict()
    res = dict()
    for k, v in df_dict['option'].items():
        imp = map(lambda x: x.replace("'", ''), 
                  df_dict['impossible_value'][k][1:-1].split()) 
        res[v] = set(imp)
    return res

# Build the dict of impossible values
impossible_values = read_imp_val_csv('examples/impossible.csv')

def get_dict(filename):
    """Read a .config like file
    :param: filename
    :type: string
    :return: dictionary associating option and its value in the file
    :rtype: dict
    """
    res = dict()
    with open(filename, 'r') as f:
        config_lines = f.readlines()
        for line in config_lines:
            if not line.startswith("#") and line != '\n':
                m = re.search('CONFIG_(\w+)=([\w"-/]+)', line)
                var_name = m.group(1)
                var_val = m.group(2)
                res[var_name] = var_val
    return res


def check_inconsistency(config_dict):
    """
    Checks if an impossible value is set for an option a dictionary
    :param: dictionary representing a .config file
    :rtype:
    """
    res = dict()
    for k in config_dict:
        if k in impossible_values:
            if config_dict[k] in impossible_values[k]:
                res[feature] = config_dict[k]
    return res


def checker(n):
    """
    Checks inconsistency n times for each generation of a .config file
    """
    def rapporteur(d):
        if d:
            print()
            for k, v in d.items():
                print('\t', k, "was set to an impossible value", v)
        else:
            print("[OK]")

    for i in range(n):
        for flag in CONF_FLAG:
            os.system("make {} -C {}".format(flag, CONFIG_DIR))
            config_dict = get_dict("{}/.config".format(CONFIG_DIR))
            print("{:15s} test {:4d} : ".format(flag.upper(), i), end="")
            inconsistence = check_inconsistency(config_dict)
            rapporteur(inconsistence)

In [6]:
checker(5)

RANDCONFIG      test    0 : [OK]
DEFCONFIG       test    0 : [OK]
ALLNOCONFIG     test    0 : [OK]
ALLYESCONFIG    test    0 : [OK]
ALLMODCONFIG    test    0 : [OK]
ALLDEFCONFIG    test    0 : [OK]
RANDCONFIG      test    1 : [OK]
DEFCONFIG       test    1 : [OK]
ALLNOCONFIG     test    1 : [OK]
ALLYESCONFIG    test    1 : [OK]
ALLMODCONFIG    test    1 : [OK]
ALLDEFCONFIG    test    1 : [OK]
RANDCONFIG      test    2 : [OK]
DEFCONFIG       test    2 : [OK]
ALLNOCONFIG     test    2 : [OK]
ALLYESCONFIG    test    2 : [OK]
ALLMODCONFIG    test    2 : [OK]
ALLDEFCONFIG    test    2 : [OK]
RANDCONFIG      test    3 : [OK]
DEFCONFIG       test    3 : [OK]
ALLNOCONFIG     test    3 : [OK]
ALLYESCONFIG    test    3 : [OK]
ALLMODCONFIG    test    3 : [OK]
ALLDEFCONFIG    test    3 : [OK]
RANDCONFIG      test    4 : [OK]
DEFCONFIG       test    4 : [OK]
ALLNOCONFIG     test    4 : [OK]
ALLYESCONFIG    test    4 : [OK]
ALLMODCONFIG    test    4 : [OK]
ALLDEFCONFIG    test    4 : [OK]
